In [ ]:
import os
from copy import deepcopy

import xarray as xr
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm

import distributed

from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import matplotlib.pyplot as plt
import matplotlib.colors as mplc
import cartopy.crs as ccrs
import matplotlib.dates as mdates
import matplotlib.lines as mpll
from pandas.plotting import register_matplotlib_converters

In [ ]:
rnd = np.random.RandomState(42)

In [ ]:
plt.style.use('paper')
plt.style.use('egu_journals')
plt.rcParams['text.usetex'] = False
register_matplotlib_converters()
rotated_pole = ccrs.RotatedPole(pole_longitude=-171.0, pole_latitude=41.5)
plate_carree = ccrs.PlateCarree()

In [ ]:
cluster = distributed.LocalCluster()
client = distributed.Client(cluster)
client

# Load data

In [ ]:
base_path = '/work/um0203/u300636/for2131/runs/da_enkf_for_soil/'

## Temp

In [ ]:
vr_t_path = os.path.join(base_path, '016', 'temp.nc')
vr_t = xr.open_dataset(vr_t_path)['T'].squeeze(drop=True).chunk((1, 1, 109, 99)).isel(level=-1)[..., 10:-10, 10:-10]

In [ ]:
det_t_path = os.path.join(base_path, '018', 'temp.nc')
det_t = xr.open_dataset(det_t_path)['T'].squeeze(drop=True).chunk((1, 1, 109, 99)).isel(level=-1)[..., 10:-10, 10:-10]

In [ ]:
sekf_t_path = os.path.join(base_path, '023', 'juwels', 'temp_cleaned.nc')
sekf_t = xr.open_dataset(sekf_t_path)['T'].squeeze(drop=True).chunk((1, 109, 99))[..., 10:-10, 10:-10]

In [ ]:
ens_t_path = os.path.join(base_path, '015', 'temp.nc')
ens_t = xr.open_dataset(ens_t_path)['T'].squeeze(drop=True).chunk((40, 1, 1, 109, 99)).isel(level=-1)[..., 10:-10, 10:-10]

In [ ]:
da_land_t_path = os.path.join(base_path, '019', 'temp.nc')
da_land_t = xr.open_dataset(da_land_t_path)['T'].squeeze(drop=True).chunk((40, 1, 1, 109, 99)).isel(level=-1)[..., 10:-10, 10:-10]

In [ ]:
da_t_path = os.path.join(base_path, '020', 'temp.nc')
da_t = xr.open_dataset(da_t_path)['T'].squeeze(drop=True).chunk((40, 1, 1, 109, 99)).isel(level=-1)[..., 10:-10, 10:-10]

In [ ]:
da_land_t['ensemble'] = ens_t['ensemble'] = da_t['ensemble'] = np.arange(40)

In [ ]:
vr_t = vr_t.sel(time=~vr_t.indexes['time'].duplicated())
det_t = det_t.sel(time=~det_t.indexes['time'].duplicated())
sekf_t = sekf_t.sel(time=~sekf_t.indexes['time'].duplicated())
ens_t = ens_t.sel(time=~ens_t.indexes['time'].duplicated())
da_land_t = da_land_t.sel(time=~da_land_t.indexes['time'].duplicated())
da_t = da_t.sel(time=~da_t.indexes['time'].duplicated())

# Error estimation

In [ ]:
err_ens_t = np.sqrt(((ens_t.mean('ensemble')-vr_t) ** 2).mean(['rlat', 'rlon'])).load()
err_da_land_t = np.sqrt(((da_land_t.mean('ensemble')-vr_t) ** 2).mean(['rlat', 'rlon'])).load()[1:]
err_da_t = np.sqrt(((da_t.mean('ensemble')-vr_t) ** 2).mean(['rlat', 'rlon'])).load()[1:]
err_det_t = np.sqrt(((det_t-vr_t) ** 2).mean(['rlat', 'rlon'])).load()
err_sekf_t = np.sqrt(((sekf_t-vr_t) ** 2).mean(['rlat', 'rlon'])).load()

# Plot

In [ ]:
fig, ax = plt.subplots()
#ax.axvline(x=pd.to_datetime('2015-07-31 12:00'), c='0.5', alpha=0.5)
ax.fill_betweenx(x1=[pd.to_datetime('2015-07-31 12:00')]*2, y=[-1, 1], color='0.8', alpha=0.35, lw=0)
ax.axvline(x=pd.to_datetime('2015-07-30 00:00'), c='black', lw=0.5, zorder=-99)
ax.axvline(x=pd.to_datetime('2015-07-31 00:00'), c='black', lw=0.5, zorder=-99)
ax.axvline(x=pd.to_datetime('2015-08-01 00:00'), c='black', lw=0.5, zorder=-99)
ax.axvline(x=pd.to_datetime('2015-08-02 00:00'), c='black', lw=0.5, zorder=-99)
ax.axvline(x=pd.to_datetime('2015-08-03 00:00'), c='black', lw=0.5, zorder=-99)
ax.axvline(x=pd.to_datetime('2015-08-04 00:00'), c='black', lw=0.5, zorder=-99)
ax.axvline(x=pd.to_datetime('2015-08-05 00:00'), c='black', lw=0.5, zorder=-99)
ax.axvline(x=pd.to_datetime('2015-08-06 00:00'), c='black', lw=0.5, zorder=-99)
ax.axvline(x=pd.to_datetime('2015-08-07 00:00'), c='black', lw=0.5, zorder=-99)

ax.plot(err_ens_t[::4].time, err_ens_t[::4], c='black', label='ENS')
ax.plot(err_det_t[::4].time, err_det_t[::4], label='DET', c='C0')
ax.plot(err_sekf_t[3+23*4::4].time, err_sekf_t[3+23*4::4].isel(level=-1), label='SEKF', c='deepskyblue')
ax.plot(err_da_land_t[::4].time, err_da_land_t[::4], label='LETKF Soil', c='salmon')
ax.plot(err_da_t[::4].time, err_da_t[::4], label='LETKF Soil+Temp', c='firebrick')

#ax.text(x=pd.to_datetime('2015-07-30 15:00'), y=0.0003, s='spin-up', ha='center', va='bottom', zorder=9999)
ax.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d'))
ax.set_xlim(pd.to_datetime('2015-07-29 18:00'), pd.to_datetime('2015-08-08 00:00'))
ax.set_xlabel('Time (UTC)')
ax.set_ylabel('RMSE to NATURE (K)')
ax.legend(loc=0, framealpha=1)
ax.set_ylim(0, 0.5)
#ax.plot(spread_t.time, spread_t, label='Ens Spread', c='firebrick')
plt.show()
fig.savefig('../figures/fig_04_err_t_long.png', dpi=300)